In [ ]:
!python -m spacy download ru_core_news_sm  # no option to install by pip

In [ ]:
import spacy
import pytextrank
from spacy.lang.ru.stop_words import STOP_WORDS
from string import punctuation

In [ ]:
stopwords = list(STOP_WORDS)
punctuation += '\n'

In [ ]:
nlp = spacy.load("ru_core_news_sm")
nlp.add_pipe("textrank", last=True)

In [ ]:
text = '''Сколько статей было написано на хабре от обучающих курсов, как тот или иной человек вошел в ИТ, несмотря на то, что он сам юрист и ему уже за 40 лет. Такие статьи вселяют надежду, как передающиеся из уст в уста рассказы о чудотворцах, которые могли исцелять одним касанием.
Не буду оспаривать реалистичность таких вхождений в ИТ... Но давайте зададимся одним вопросом, а какое соотношение таких "чудо-людей" к обычным смертным - тем, кто не прошел в ИТ? Для этого достаточно почитать про понятие "систематическая ошибка выжившего", но тому, кому этого не достаточно, давайте вместе разберемся в цифрах.
Отовсюду мы слышим одно и тоже: нехватка ИТ специалистов, высокие зарплаты, удаленная работа, поддержка ИТ на уровне государства. Звучит шикарно. Возникает ощущение, что это бездонная сфера, которая будет востребована, ну хотя бы вечность, не меньше.
Отступление: ничто не вечно... Инженеры, юристы, экономисты - всех их объединяет, то что они имели высокую популярность в свое время, но время это ушло...
Давайте посмотрим немного повнимательнее на эту сферу ИТ, выбросим всю эту СМИ мишуру, снимем пару ведер лапши с ушей.
У меня есть знакомые, которые хотели войти в ИТ, но так и не вошли. Мое личное соотношение примерно 1 вошедший на 7 не вошедших. Конкуренция очень жесткая.'''

In [ ]:
doc = nlp(text)

In [ ]:
# var 1
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies:
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [ ]:
# var 2
for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=2):
    print(sent)